In [9]:
import pandas as pd
import numpy as np
import time
import fpGrowth 

In [2]:
#预处理数据
def pre_data(df):
    df['timestamp'] = pd.to_datetime(df['sldat'])
    
    #分别按vipno、sldat排序
    data = df.sort_values(by=['vipno', 'sldat'])
    
    #分组
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0, method='first')
    
    #取每个vipno的前60%的数据
    data = data.groupby(['vipno'], as_index = False).apply(lambda x: x[x['rank'] <=  round(0.6 * x['rank'].max())])
    #整理
    data = data.drop(['timestamp', 'rank'], axis = 1).reset_index(drop=True)
    return data

In [3]:

#提出bndno的数据
def pre_bndno(data):
    #丢弃多余的列
    data_bndno = data.drop(['sldat', 'vipno', 'pluno', 'dptno'], axis = 1)
    
    #合并订单
    data_bndno['value'] = data_bndno['bndno']
    data_bndno = data_bndno.pivot_table(data_bndno , index = ['uid'], columns = 'bndno')
    
    #整理
    data_bndno = data_bndno.fillna(0).transpose().loc['value'].transpose()
    del data_bndno.index.name
    del data_bndno.transpose().index.name
    
    #将dataframe转为array
    array_bndno = []
    for row in data_bndno.as_matrix():
        array_bndno.append([x for x in row if x != 0.0])
    return array_bndno

In [7]:

def fp_bndno(data):
    thresholds = [2, 4, 8, 16, 32, 64]
    array_bndno = pre_bndno(data)
    for n in thresholds:
        freq_sets = fpGrowth.fpGrowth(array_bndno, n)
        print("for bndno, threshold: ", n)
        for k in freq_sets:
            if len(k) >= 2:
                print (k, freq_sets[k])

In [10]:
if __name__ == "__main__":
    old_df = pd.read_csv('../trade.csv', usecols=['uid', 'vipno', 'sldat', 'pluno', 'dptno', 'bndno'])
    #旧数据
    old_data = pre_data(old_df)
    start = time.clock()
    fp_bndno(old_data)
    time_old = time.clock() - start;
    
    #新数据
    new_df = pd.read_csv('../trade_new.csv', usecols=['uid', 'vipno', 'sldatime', 'pluno', 'dptno', 'bndno'])
    new_df.rename(columns={ new_df.columns[1]: "sldat" }, inplace=True)    
    new_data = pre_data(new_df)
    start = time.clock()
    fp_bndno(new_data)
    time_new = time.clock() - start;
    
    print("fp_bndno,for old data: ", time_old)
    print("fp_bndno,for new data: ", time_new)

for bndno, threshold:  2
frozenset({14362.0, 14123.0}) 2
frozenset({14355.0, 30253.0}) 2
frozenset({14393.0, 14314.0}) 2
frozenset({14442.0, 14445.0}) 2
frozenset({30248.0, 15003.0}) 2
frozenset({30248.0, 14010.0}) 2
frozenset({14810.0, 14301.0}) 2
frozenset({30248.0, 14703.0}) 2
frozenset({14753.0, 14703.0}) 2
frozenset({30248.0, 14539.0}) 2
frozenset({14539.0, 14622.0}) 2
frozenset({30248.0, 14539.0, 14622.0}) 2
frozenset({14203.0, 14366.0}) 2
frozenset({34003.0, 15039.0}) 2
frozenset({30248.0, 11291.0}) 2
frozenset({10068.0, 10703.0}) 2
frozenset({10154.0, 10086.0}) 2
frozenset({14784.0, 15052.0}) 2
frozenset({14784.0, 15094.0}) 2
frozenset({14784.0, 15052.0, 15094.0}) 2
frozenset({14784.0, 14785.0}) 2
frozenset({14785.0, 15052.0}) 2
frozenset({14784.0, 14785.0, 15052.0}) 2
frozenset({14785.0, 15094.0}) 2
frozenset({14784.0, 14785.0, 15094.0}) 2
frozenset({14785.0, 15052.0, 15094.0}) 2
frozenset({14784.0, 14785.0, 15052.0, 15094.0}) 2
frozenset({14753.0, 14279.0}) 2
frozenset({30248

frozenset({30248.0, 15066.0}) 5
frozenset({30248.0, 14810.0}) 4
frozenset({30248.0, 30170.0}) 6
frozenset({15052.0, 14622.0}) 5
frozenset({30248.0, 14622.0}) 9
frozenset({15052.0, 15038.0}) 4
frozenset({15038.0, 15631.0}) 4
frozenset({30248.0, 15038.0}) 7
frozenset({14160.0, 30248.0}) 4
frozenset({14160.0, 15052.0}) 4
frozenset({14160.0, 15094.0}) 5
frozenset({30248.0, 14196.0}) 11
frozenset({15092.0, 15039.0}) 4
frozenset({15052.0, 15092.0}) 5
frozenset({15092.0, 15094.0}) 6
frozenset({15012.0, 15092.0}) 7
frozenset({15009.0, 15092.0}) 7
frozenset({30248.0, 15092.0}) 11
frozenset({10060.0, 10086.0}) 4
frozenset({10060.0, 15631.0}) 5
frozenset({30248.0, 10060.0}) 11
frozenset({15052.0, 10086.0}) 4
frozenset({10086.0, 15631.0}) 4
frozenset({15012.0, 10086.0}) 4
frozenset({10086.0, 15039.0}) 5
frozenset({15094.0, 10086.0}) 6
frozenset({30248.0, 10086.0}) 19
frozenset({15009.0, 15052.0}) 5
frozenset({15009.0, 15094.0}) 7
frozenset({15009.0, 15631.0}) 8
frozenset({30248.0, 15009.0}) 11
fro

frozenset({10091.0, 15039.0}) 2
frozenset({30216.0, 15094.0}) 2
frozenset({30216.0, 15052.0}) 2
frozenset({30216.0, 30248.0}) 2
frozenset({30216.0, 30170.0}) 3
frozenset({14338.0, 14357.0}) 2
frozenset({30248.0, 14357.0}) 2
frozenset({14333.0, 14357.0}) 3
frozenset({14357.0, 14319.0}) 3
frozenset({14145.0, 15009.0}) 2
frozenset({14145.0, 15012.0}) 2
frozenset({30248.0, 14145.0, 15012.0}) 2
frozenset({14145.0, 15094.0}) 2
frozenset({14145.0, 14098.0}) 2
frozenset({14168.0, 14145.0}) 2
frozenset({30248.0, 14145.0}) 3
frozenset({15577.0, 15083.0}) 2
frozenset({15577.0, 15039.0}) 2
frozenset({15577.0, 14622.0, 15039.0}) 2
frozenset({14168.0, 15577.0}) 2
frozenset({14168.0, 15577.0, 14622.0}) 2
frozenset({15577.0, 14330.0}) 2
frozenset({15577.0, 14622.0}) 3
frozenset({30248.0, 15577.0, 14622.0}) 2
frozenset({30248.0, 15577.0}) 4
frozenset({14152.0, 10107.0}) 2
frozenset({14152.0, 15039.0}) 2
frozenset({14152.0, 15012.0}) 2
frozenset({14152.0, 15094.0}) 2
frozenset({14152.0, 14442.0}) 2
froz

frozenset({15083.0, 15094.0}) 3
frozenset({30248.0, 15083.0, 15094.0}) 2
frozenset({15066.0, 15083.0}) 3
frozenset({15066.0, 15083.0, 15045.0}) 2
frozenset({30248.0, 15066.0, 15083.0, 15045.0}) 2
frozenset({30248.0, 15066.0, 15083.0}) 3
frozenset({15083.0, 15045.0}) 3
frozenset({30248.0, 15083.0, 15045.0}) 3
frozenset({15009.0, 15083.0}) 4
frozenset({15083.0, 15052.0}) 5
frozenset({30248.0, 15083.0, 15052.0}) 2
frozenset({30248.0, 15083.0}) 12
frozenset({14177.0, 14319.0}) 2
frozenset({30170.0, 14319.0}) 2
frozenset({15011.0, 14319.0}) 2
frozenset({15009.0, 14319.0}) 2
frozenset({15012.0, 14319.0}) 2
frozenset({14810.0, 14319.0}) 3
frozenset({14475.0, 14319.0}) 3
frozenset({14098.0, 14319.0}) 4
frozenset({15052.0, 14319.0}) 4
frozenset({30248.0, 15052.0, 14319.0}) 2
frozenset({30248.0, 14319.0}) 8
frozenset({14160.0, 14268.0}) 2
frozenset({14268.0, 15631.0}) 2
frozenset({14322.0, 14268.0}) 3
frozenset({14196.0, 14322.0, 14268.0}) 2
frozenset({14268.0, 15039.0}) 3
frozenset({14196.0, 14

frozenset({30248.0, 14301.0}) 6
frozenset({30248.0, 14333.0}) 5
frozenset({30248.0, 10033.0}) 4
frozenset({30248.0, 14805.0}) 4
frozenset({14196.0, 14740.0}) 4
frozenset({14322.0, 14740.0}) 4
frozenset({30248.0, 14740.0}) 8
frozenset({30248.0, 14191.0}) 7
frozenset({30248.0, 14386.0}) 4
frozenset({15001.0, 15094.0}) 4
frozenset({15001.0, 15012.0}) 4
frozenset({30248.0, 15001.0}) 6
frozenset({15001.0, 15052.0}) 6
frozenset({30248.0, 10659.0}) 7
frozenset({30248.0, 14545.0}) 5
frozenset({30248.0, 10045.0}) 7
frozenset({30248.0, 14123.0}) 7
frozenset({15052.0, 14213.0}) 4
frozenset({14213.0, 14079.0}) 4
frozenset({30248.0, 14213.0}) 5
frozenset({14810.0, 14213.0}) 6
frozenset({30248.0, 14231.0}) 6
frozenset({30248.0, 15002.0}) 4
frozenset({30248.0, 30833.0}) 5
frozenset({15073.0, 15009.0}) 6
frozenset({30248.0, 15073.0}) 7
frozenset({15057.0, 15094.0}) 4
frozenset({30248.0, 15057.0}) 9
frozenset({30248.0, 10094.0}) 6
frozenset({30248.0, 14079.0}) 8
frozenset({14032.0, 14177.0}) 4
frozense